In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel

/Users/lucanyckees/Desktop/my-repos/recommendation_system/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
# Define a simple dataset with triplets (anchor, positive, negative text)
class TripletTextDataset(Dataset):
    def __init__(self, anchor_texts, positive_texts, negative_texts, tokenizer, max_len=32):
        self.anchor_texts = anchor_texts
        self.positive_texts = positive_texts
        self.negative_texts = negative_texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.anchor_texts)

    def __getitem__(self, idx):
        anchor_text = self.anchor_texts[idx]
        positive_text = self.positive_texts[idx]
        negative_text = self.negative_texts[idx]

        anchor = self.tokenizer(anchor_text, padding='max_length', truncation=True, max_length=self.max_len, return_tensors="pt")
        positive = self.tokenizer(positive_text, padding='max_length', truncation=True, max_length=self.max_len, return_tensors="pt")
        negative = self.tokenizer(negative_text, padding='max_length', truncation=True, max_length=self.max_len, return_tensors="pt")

        return anchor, positive, negative
    

# Pretrained BERT model for embeddings
class BertEmbeddingNet(nn.Module):
    def __init__(self, pretrained_model_name='bert-base-uncased'):
        super(BertEmbeddingNet, self).__init__()
        self.bert = BertModel.from_pretrained(pretrained_model_name)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # Use the CLS token's output as the sentence embedding
        return output.last_hidden_state[:, 0, :]  # [batch_size, hidden_dim]


# Triplet loss function
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.loss_fn = nn.TripletMarginLoss(margin=margin)

    def forward(self, anchor, positive, negative):
        return self.loss_fn(anchor, positive, negative)

In [18]:
# Prepare data (example placeholders)
anchor_texts = ["king", "queen", "paris"]
positive_texts = ["monarch", "monarch", "france"]
negative_texts = ["dog", "cat", "germany"]

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Dataset and DataLoader
dataset = TripletTextDataset(anchor_texts, positive_texts, negative_texts, tokenizer, max_len=8)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [20]:
dataset.__getitem__(0)

({'input_ids': tensor([[ 101, 2332,  102,    0,    0,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 0, 0, 0, 0, 0]])},
 {'input_ids': tensor([[  101, 11590,   102,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 0, 0, 0, 0, 0]])},
 {'input_ids': tensor([[ 101, 3899,  102,    0,    0,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 0, 0, 0, 0, 0]])})

In [21]:
# Initialize the network, loss function, and optimizer
embedding_net = BertEmbeddingNet(pretrained_model_name='bert-base-uncased')

In [23]:
triplet_loss_fn = TripletLoss(margin=1.0)

In [24]:
optimizer = optim.Adam(embedding_net.parameters(), lr=1e-5)

In [25]:
# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    embedding_net.train()
    running_loss = 0.0
    for anchors, positives, negatives in dataloader:
        optimizer.zero_grad()

        # Process anchors
        anchor_input_ids = anchors['input_ids'].squeeze(1)  # Remove extra batch dimension
        anchor_attention_mask = anchors['attention_mask'].squeeze(1)
        anchor_embeddings = embedding_net(anchor_input_ids, anchor_attention_mask)

        # Process positives
        positive_input_ids = positives['input_ids'].squeeze(1)
        positive_attention_mask = positives['attention_mask'].squeeze(1)
        positive_embeddings = embedding_net(positive_input_ids, positive_attention_mask)

        # Process negatives
        negative_input_ids = negatives['input_ids'].squeeze(1)
        negative_attention_mask = negatives['attention_mask'].squeeze(1)
        negative_embeddings = embedding_net(negative_input_ids, negative_attention_mask)

        # Compute the triplet loss
        loss = triplet_loss_fn(anchor_embeddings, positive_embeddings, negative_embeddings)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Update loss
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")

Epoch [1/5], Loss: 3.3852
Epoch [2/5], Loss: 0.3529
Epoch [3/5], Loss: 2.0858
Epoch [4/5], Loss: 0.9439
Epoch [5/5], Loss: 0.8659
